# Wizeline task for bootcamp

### Carla Garcia

# Import libraries

In [2]:
import pandas as pd

# Load data

In [4]:
dataset=pd.read_csv('.\data\sample.csv')

In [13]:
#Extraction
COLUMNS_Q1=['cadenaComercial']
dataset_Q1=dataset['cadenaComercial']
#Transformation
count_cadena_comercial_Q1=dataset_Q1.nunique()

In [79]:
#Extraction
COLUMNS_Q2=['estado','producto']
dataset_Q2=dataset[COLUMNS_Q2]
#Transformation
top_10=dataset_Q2.groupby('estado').head(10).reset_index(drop=True)
top_10=dataset_Q2.sort_values('producto',ascending = False).groupby('estado').head(10)
top_10_products=dataset_Q2.groupby(COLUMNS_Q2)['producto'].count()
top_10_products_filter=top_10_products.groupby("estado", group_keys=False).nlargest(10)

In [111]:
#Extraction
COLUMNS_Q3=['cadenaComercial','producto']
dataset_Q3=dataset[COLUMNS_Q3]
#Transformation
cadena_per_product=dataset_Q3.groupby(['cadenaComercial']).producto.count()
cadena_most_monitoring_prod=cadena_per_product.nlargest(1).reset_index()
cadena_most_monitoring_prod['cadenaComercial'][0:]

0    COMERCIAL MEXICANA
Name: cadenaComercial, dtype: object

# Q1 How many commercial chains are monitored, and therefore, included in this database?


In [80]:
print(f"There are {count_cadena_comercial_Q1} commercial chains which products are  monitored")

There are 4 commercial chains which products are  monitored


#  Q2 What are the top 10 monitored products by State?

In [82]:
path=r'.\top_10_per_state.csv'
top_10_products_filter.to_csv(path)
print(f"Top 10 products by state can be reviewed in {path}")

Top 10 products by state can be reviewed in .\top_10_per_state.csv


# Q3 Which is the commercial chain with the highest number of monitored products?

In [117]:
print(f" Comercial chain with highest number of monitored products is: {cadena_most_monitoring_prod['cadenaComercial'].values[0]}")

 Comercial chain with highest number of monitored products is: COMERCIAL MEXICANA


# Q4 Use the data to find an interesting fact

# Q5 What are the lessons learned from this excersice?

# Q6 Can you identify other ways to approach this problem? Explain.